![](https://import.cdn.thinkific.com/220744/BExaQBPPQairRWFqxFbK_logo_mastermind_web_png)

¡Ahora es **vuestro turno!**

En este proyecto final os he adjuntado un Dataset con 27.075 juegos en **Steam**, la popular plataforma de juegos de PC. Esta lista está actualizada desde el inicio hasta Abril de 2019.

Ya que tenéis los conocimientos, os pediré que me contestéis a **10 preguntas**:
1. Cual es el TOP10 de juegos más jugados?
2. Cuál es el género de videojuegos más vendido?
3. Qué géneros tienen mejores reviews de media?
4. Cuánto ha ganado el desarrollador que más juegos ha vendido?
5. Cómo afecta a las reviews que un juego sea Early Access?
6. Cuál es el desarrollador (o desarrolladores) que más tipos de juego y cantidad ha desarrollado?
7. En qué mes, desde que tenemos datos, se han publicado más juegos?
8. Cuántas horas se juega de media a cada tipo de juego mensualmente?
9. Los juegos Free to Play tienen mejores o peores reviews?
10. Cómo ha evolucionado el número de achievements (trofeos) por juego?

Os dejamos via libre para contestar a estas preguntas como más os apetezca. Recordad que no hay nada de malo en consultar por internet si tenéis dudas. Las documentaciones oficiales o [Stack Overflow](https://stackoverflow.com/) son sitios geniales para ello!

Al acabar este proyecto, publicadlo en el foro donde está colgado para ver quién es el mejor!

Os dejo la guía de pasos a seguir.

Muchos ánimos!!




# Importando librerías

In [1]:
import urllib.request as Req
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from thefuzz import process,fuzz
from unidecode import unidecode
import pdb

%matplotlib inline

# Consiguiendo los datos

### Conseguir link

In [2]:
url = 'https://neo-mastermind.s3.amazonaws.com/uploads/froala_editor/files/SteamDataset-220424-201858.zip'

### Descargar el archivo

In [3]:
Req.urlretrieve(url, 'steam.zip')

('steam.zip', <http.client.HTTPMessage at 0x1c1dd44f0b0>)

### Descomprimir el archivo

In [4]:
steam_zip = ZipFile('steam.zip')

In [5]:
steam_zip.filelist

[<ZipInfo filename='steam.csv' compress_type=deflate external_attr=0x20 file_size=3425223 compress_size=979195>]

In [6]:
archivo = steam_zip.open('steam.csv')

### Construir DataFrame

In [7]:
df = pd.read_csv(archivo, index_col=0)

# Limpieza y manipulación de datos

## Quitar datos duplicados o irrelevantes

In [8]:
df.duplicated().any()

True

In [9]:
df.duplicated().sum()

28

In [10]:
df = df.drop_duplicates()

In [11]:
df.columns

Index(['name', 'release_date', 'developer', 'achievements', 'positive_ratings',
       'negative_ratings', 'avg_hours_per_user', 'price', 'sells', 'action',
       'adventure', 'rpg', 'simulation', 'strategy', 'racing', 'free_to_play',
       'early_access'],
      dtype='object')

In [12]:
df.drop('free_to_play', axis=1, inplace=True) # Redundante si tenemos la columna price, con la cual también generaba disparidades

## Arreglar errores estructurales

In [13]:
df.head()

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,early_access
0,Counter-Strike,2000-11-01,Valve,0,124534,3339,17612,7.19€,15000000.0,True,False,False,False,False,False,False
1,Team Fortress Classic,1999-04-01,Valve,0,3318,633,277,3.99€,7500000.0,True,False,False,False,False,False,False
2,Day of Defeat,2003-05-01,Valve,0,3416,398,187,3.99€,7500000.0,True,False,False,False,False,False,False
3,Deathmatch Classic,2001-06-01,Valve,0,1273,267,258,3.99€,7500000.0,True,False,False,False,False,False,False
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,0,5250,288,624,3.99€,7500000.0,True,False,False,False,False,False,False


In [14]:
df.tail()

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,early_access
27070,Room of Pandora,2019-04-24,SHEN JIAWEI,7,3,0,0,2.09€,10000.0,False,True,False,False,False,False,False
27071,Cyber Gun,2019-04-23,Semyon Maximov,0,8,1,0,1.69€,10000.0,True,True,False,False,False,False,False
27072,Super Star Blast,2019-04-24,EntwicklerX,24,0,1,0,3.99€,10000.0,True,False,False,False,False,False,False
27073,New Yankee 7: Deer Hunters,2019-04-17,Yustas Game Studio,0,2,0,0,5.19€,10000.0,False,True,False,False,False,False,False
27074,Rune Lord,2019-04-24,Adept Studios GD,0,4,0,0,5.19€,10000.0,False,True,False,False,False,False,False


In [15]:
df.sample(5)

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,early_access
18466,The adventure of Kroos,2017-12-06,M.J.H.X. Studio,30,14,6,0,6.19€,10000.0,True,True,True,False,False,False,True
24145,Sexy Jigsaw / Sexy Puzzle / 性感拼图,2018-08-24,ZOV GAME STUDIO,18,68,8,0,2.89€,10000.0,False,False,False,False,True,False,False
4216,Towers of Altrac - Epic Defense Battles,2015-01-05,Jsmael Stieger;Manuel Stieger;Andreas Stieger,82,63,44,2,2.09€,10000.0,True,False,False,False,True,False,False
11653,Operation Breakout,2016-10-28,SGTTB_Studios;Phanom Games,4,34,47,292,5.99€,75000.0,True,True,False,False,False,False,False
25634,Laser Party,2018-12-28,Cross Link,50,18,4,0,1.69€,10000.0,True,False,False,False,False,False,False


### Definir errores

Recuerda hacer una exploración de datos para encontrar estos errores. Os confirmo que hay unos cuantos!

Tanto "price" como "sells" intenté reasignarlos con df[columna].astype(type) para evitar el warning, pero en ambos casos no funcionó y se mantenía en tipo object, por lo que hice la asignación normal con df[columna] = df[columna].astype(type)

#### Separar múltiples desarrolladoras en varias listas

In [16]:
df.loc[:, 'developer'] = df['developer'].str.split(";")

In [17]:
df = df.explode('developer').reset_index(drop=True)

#### Eliminar carácteres especiales de "developer"

In [18]:
pat = r'[^\w\s]|_'

In [19]:
df.loc[:, 'developer'] = df['developer'].str.replace(pat, ' ', regex=True)

In [20]:
df.loc[:, 'developer'] = df['developer'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [21]:
df.loc[:, 'developer'] = df['developer'].str.replace(r'\s+', ' ', regex=True)

In [22]:
df.loc[22980, 'developer'] = df.loc[22981, 'developer']

In [23]:
df = df.drop_duplicates()

#### Eliminar espacios extras en "name" y "developer"

In [24]:
name_espacios = df['name'] != df['name'].str.strip()

In [25]:
name_espacios.unique()

array([False,  True])

In [26]:
df.loc[:, 'name'] = df['name'].str.strip()

In [27]:
developer_espacios = df['developer'] != df['developer'].str.strip()

In [28]:
developer_espacios.unique()

array([False,  True])

In [29]:
df.loc[:, 'developer'] = df['developer'].str.strip()

In [30]:
df[df['developer'].str.strip() != df['developer']]

,name,release_date,developer,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price,sells,action,adventure,rpg,simulation,strategy,racing,early_access
25003,The Battle Of Ages,2018-09-07,NaN,0,2,1,0,3.99€,10000.0,True,False,True,False,False,False,True


#### Normalizar nombres en "developer"

##### Intento Fallido

Intenté crear un diccionario de equivalencias que afecte a todas las desarrolladoras en el DataFrame, le di muchas vueltas al problema para poder solucionarlo, pero no pude. Son demasiadas desarrolladoras distintas, muchas con nombres muy parecidos entre si, pero que no son una misma desarrolladora, o con palabras en común que dificultaba la diferenciación entre las que pertenecen y la que no a una misma desarrolladora. Dejo igualmente el código que estuve probando, del cual agradecería tener una correción y alguna solución a este problema, ya que no quedé contento teniendo que resolver este problema a medias, de ser posible me gustaría poder arreglarlo aunque sea posterior a la correción incluso. Muchas gracias!

In [31]:
"""devs = set(df['developer'].unique())
devs_lower = {dev.lower(): dev for dev in devs if isinstance(dev, str)}
matches = {}
processed = set()"""

"devs = set(df['developer'].unique())\ndevs_lower = {dev.lower(): dev for dev in devs if isinstance(dev, str)}\nmatches = {}\nprocessed = set()"

In [32]:
"""for dev in devs:
    if dev in processed or pd.isna(dev):
        continue
    close_matches = process.extract(dev.lower(), devs_lower.keys(), limit=10, scorer=fuzz.ratio)
    group = [name for name, score in close_matches if score > 85]
    
    if len(set(group)) > 1:
        reference = min(group, key=len)
        for name in group:
            matches[devs_lower[name]] = devs_lower[reference]
        processed.update(group)"""

'for dev in devs:\n    if dev in processed or pd.isna(dev):\n        continue\n    close_matches = process.extract(dev.lower(), devs_lower.keys(), limit=10, scorer=fuzz.ratio)\n    group = [name for name, score in close_matches if score > 85]\n    \n    if len(set(group)) > 1:\n        reference = min(group, key=len)\n        for name in group:\n            matches[devs_lower[name]] = devs_lower[reference]\n        processed.update(group)'

##### Desarrolladoras más importante

Cree este diccionario de forma manual para solucionar al menos una parte del problema y poder continuar con el trabajo. Se que no es lo óptimo, pero es lo más que pude hacer, realmente esta parte de normalización de las desarrolladoras me excedió.

In [33]:
developer_mapping = {
    'CAPCOM Co Ltd': 'Capcom', 
    'Capcom': 'Capcom', 
    'CAPCOM CO LTD': 'Capcom', 
    'Capcom Vancouver': 'Capcom', 
    'Capcom U S A Inc': 'Capcom', 
    'Capcom Game Studio Vancouver': 'Capcom', 
    'CAPCOM': 'Capcom', 
    'Capcom Game Studio Vancouver Inc': 'Capcom',
    'Ubisoft': 'Ubisoft', 
    'Ubisoft Montreal': 'Ubisoft', 
    'Ubisoft Bulgaria': 'Ubisoft', 
    'Ubisoft Montpellier': 'Ubisoft', 
    'Ubisoft Romania': 'Ubisoft', 
    'Ubisoft Reflections': 'Ubisoft', 
    'Ubisoft Shanghai': 'Ubisoft', 
    'Ubisoft San Francisco': 'Ubisoft', 
    'Ubisoft Paris': 'Ubisoft', 
    'Ubisoft Montreal Massive Entertainment and Ubisoft Shanghai': 'Ubisoft', 
    'Ubisoft Toronto': 'Ubisoft', 
    'Ivory Tower in collaboration with Ubisoft Reflections': 'Ubisoft', 
    'RedLynx in collaboration with Ubisoft Shanghai Ubisoft Kiev': 'Ubisoft', 
    'Ubisoft Sofia': 'Ubisoft', 
    'Ubisoft Montreal Red Storm Shanghai Toronto Kiev': 'Ubisoft', 
    'Ubisoft Quebec': 'Ubisoft', 
    'Ubisoft Kiev': 'Ubisoft', 
    'Reflections a Ubisoft Studio': 'Ubisoft', 
    'Ubisoft Quebec in collaboration with Ubisoft Annecy Bucharest Kiev Montreal Montpellier Shanghai Singapore Sofia Toronto studios': 'Ubisoft', 
    'Ubisoft Montreal Studio': 'Ubisoft', 
    'Ubisoft Pune': 'Ubisoft', 
    'Ubisoft Annecy': 'Ubisoft', 
    'Ubisoft Bucharest': 'Ubisoft', 
    'Ubisoft Milan': 'Ubisoft', 
    'Ubisoft Belgrade': 'Ubisoft', 
    'Ubisoft Entertainment': 'Ubisoft', 
    'Ubisoft Singapore': 'Ubisoft', 
    'Ubisoft Blue Byte': 'Ubisoft',
    'Valve': 'Valve', 
    'Valve Corporation Nexon Korea Corporation': 'Valve',
    'BANDAI NAMCO Studio Inc': 'Bandai Namco', 
    'NAMCO BANDAI Games': 'Bandai Namco', 
    'BANDAI NAMCO Entertainment Inc': 'Bandai Namco', 
    'BANDAI NAMCO Studios Inc': 'Bandai Namco', 
    'BANDAI NAMCO Studio': 'Bandai Namco', 
    'BANDAI NAMCO Studios Vancouver': 'Bandai Namco', 
    'BANDAI NAMCO Studios': 'Bandai Namco',
    'Square Enix': 'Square Enix', 
    'SQUARE ENIX': 'Square Enix', 
    'Square Enix Montreal': 'Square Enix',
    'SEGA': 'SEGA', 
    'SEGA Studios Australia': 'SEGA', 
    'SEGA Hardlight': 'SEGA',
    '2K Czech': '2K', 
    '2K Marin': '2K', 
    '2K Australia': '2K', 
    '2K Boston': '2K', 
    '2K China': '2K',
    'FromSoftware': 'FromSoftware', 
    'FromSoftware Inc': 'FromSoftware',
    'BioWare Corporation': 'BioWare', 
    'BioWare': 'BioWare'
}

In [34]:
df['developer'] = df['developer'].replace(developer_mapping)

In [35]:
df = df.drop_duplicates()

In [36]:
len(df)

29285

#### Convertir "release_date" a datetime

In [37]:
df['release_date'] = pd.to_datetime(df['release_date'])

#### Quitar € de "price" y pasarlo a float

In [38]:
df.rename(columns={'price': 'price €'}, inplace=True)

In [39]:
df['price €'] = df['price €'].str.replace("€", "").astype(float)

#### Convertir "sells" a int

In [40]:
df['sells'] = df['sells'].astype('int64')

## Filtrar outliers 

In [41]:
df.describe()

,release_date,achievements,positive_ratings,negative_ratings,avg_hours_per_user,price €,sells
count,29285,29285.000000,2.928500e+04,29285.000000,29285.000000,29285.000000,2.928500e+04
mean,2016-12-31 03:47:34.061806592,43.449411,1.163887e+03,241.406829,166.173912,6.161731,1.436638e+05
min,1997-06-30 00:00:00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000e+04
25%,2016-03-30 00:00:00,0.000000,7.000000e+00,2.000000,0.000000,1.690000,1.000000e+04
50%,2017-08-02 00:00:00,7.000000,2.500000e+01,9.000000,0.000000,3.990000,1.000000e+04
75%,2018-06-06 00:00:00,23.000000,1.350000e+02,43.000000,0.000000,7.190000,3.500000e+04
max,2019-05-01 00:00:00,9821.000000,2.644404e+06,487076.000000,190625.000000,421.990000,1.500000e+08
std,NaN,340.558941,2.409716e+04,4809.729389,2238.965775,7.950968,1.369494e+06


Decidí únicamente definir en variables los límites dentro de los cuales los valores son considerados normales para cada columna, y así luego filtrar mediante estas variables las columnas a la hora de hacer las gráficas. Esto con el fin de no eliminar filas (juegos), que puedan después alterar las mediciones cuantitatívas como podría ser la respuesta a la pregunta: *Cuál es el desarrollador (o desarrolladores) que más tipos de juego y cantidad ha desarrollado?*

In [42]:
bounds = []
for column in ['achievements', 'positive_ratings', 'negative_ratings', 'price €', 'sells']:
    q1 = df.describe().loc['25%', column]
    q3 = df.describe().loc['75%', column]
    iqr = q3 - q1
    
    upper_bound = q3 + 1.5 * iqr
    bounds.append(round(upper_bound, 2))
    
achievements_bound, positive_ratings_bound, negative_ratings_bound, price_bound, sells_bound = bounds
avg_bound = 385

De la totalidad de los juegos, **22435** no tienen ni una hora jugada, los otros **6850** si. La media de horas jugadas de la totalidad de los juegos es de **166 horas**, y la media considerando solamente los juegos que tienen horas jugadas es de **710 horas**.

Calculando los limites dentro de los cuales los valores serían considerados normales con desviación estándar, ya que todos los cuartiles son iguales a cero, solo quedarían aquellos juegos por debajo de las **6883 horas**, lo cual representa un **4118%** más que la media incluyendo todos los juegos y un **869%** más que la media de 710 horas, y equivale a **6765 juegos**, lo cual representa un **98,7%** de los juegos con horas jugadas.

En cambio, calculando los limites dentro de los cuales los valores serían considerados normales mediante un promedio de crecimiento de los valores de las demás columnas hasta su límite, las cuales si pudieron ser calculadas con sus cuartiles, solo quedarían aquellos juegos por debajo de las **385 horas**, lo cual representa un **131,7%** más que la media incluyendo todos los juegos y un **45,8%** menos que el promedio de **710 horas**, y equivale a **5203 juegos**, lo cual representa un **75.9%** de la totalidad de los juegos con horas jugadas.

Este el el razonamiento que utilicé para definir los juegos que serán considerados a la hora de utlilizar la columna "avg_hours_per_user"

## Lidiar con NANs

In [43]:
df.isna().any()

name                  False
release_date          False
developer              True
achievements          False
positive_ratings      False
negative_ratings      False
avg_hours_per_user    False
price €               False
sells                 False
action                False
adventure             False
rpg                   False
simulation            False
strategy              False
racing                False
early_access          False
dtype: bool

In [44]:
df.loc[25003, 'developer'] = 'None'

## Validar nuestros datos

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29285 entries, 0 to 29322
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                29285 non-null  object        
 1   release_date        29285 non-null  datetime64[ns]
 2   developer           29285 non-null  object        
 3   achievements        29285 non-null  int64         
 4   positive_ratings    29285 non-null  int64         
 5   negative_ratings    29285 non-null  int64         
 6   avg_hours_per_user  29285 non-null  int64         
 7   price €             29285 non-null  float64       
 8   sells               29285 non-null  int64         
 9   action              29285 non-null  bool          
 10  adventure           29285 non-null  bool          
 11  rpg                 29285 non-null  bool          
 12  simulation          29285 non-null  bool          
 13  strategy            29285 non-null  bool          


In [46]:
df.duplicated().any()

False

In [47]:
name_espacios = df['name'] != df['name'].str.strip()
name_espacios.unique()

array([False])

In [48]:
developer_espacios = df['developer'] != df['developer'].str.strip()
developer_espacios.unique()

array([False])

In [49]:
df.isna().any()

name                  False
release_date          False
developer             False
achievements          False
positive_ratings      False
negative_ratings      False
avg_hours_per_user    False
price €               False
sells                 False
action                False
adventure             False
rpg                   False
simulation            False
strategy              False
racing                False
early_access          False
dtype: bool

# EDA

## Cual es el TOP10 de juegos más jugados?

## Cuál es el género de videojuegos más vendido?

## Qué géneros tienen mejores reviews de media?

## Cuánto ha ganado el desarrollador que más juegos ha vendido?

## Cómo afecta a las reviews que un juego sea Early Access?

## Cuál es el desarrollador (o desarrolladores) que más tipos de juego y cantidad ha desarrollado?

## En qué mes, desde que tenemos datos, se han publicado más juegos?

## Cuántas horas se juega de media a cada tipo de juego mensualmente?

## Los juegos Free to Play tienen mejores o peores reviews?

## Cómo ha evolucionado el número de achievements (trofeos) por juego?

## Bonus! Si tienes curiosidad, y quieres contestar más preguntas, puedes resolverlas aquí mismo!